###

# Deep Learning (RNN) Demo for Load Forecasting

### Step 1: Import all the packages needed

In [1]:
import tensorflow as tf
from tensorflow.models.rnn import rnn, rnn_cell
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib as mp
import random as rd
import argparse
import os, sys
import csv
import math
import time
import matplotlib.pyplot as pl

### Step 2: setting all global parameters -- sec 2 network configuration

In [2]:
time1 = time.time() # set up counter to record run time
data_dir = './data/' # directory contains input data
num_epoches = 600 # training epoches for each customer samples
n_steps = 48 # input size
test_batch_size = 70*48 # days of a batch
train_batch_size = 70*48
feature_size = 1 # same time of a week
n_hidden = 10 # input size
num_layers = 2
n_output = 1
Rs = 10

### Step 4: define data generating function code. 
which generate a batch of batch-size large sequence data. the data is feature_size dims width and is a time series of float32 of steps steps. inputs and outputs are:

inputs:
----n_batch: number of samples in a batch
----steps: the sequence length of a sample data
----feature_size: dimensions of a single time step data frame

outputs:
----X inputs, shape(n_batch,steps,feature_size)
----Y outputs should be, shape(n_batch,)

In [3]:
def train_data_gen(totaltraindays,x_data,y_data,steps = 48, n_batch = train_batch_size):
    X = np.zeros((n_batch,steps,feature_size))
    Y = np.zeros((n_batch,feature_size))
    rang = range(totaltraindays) # test day sample range
    train_days_list = rd.sample(rang,n_batch) # pick unduplicated n indexes as examples
    #print totaltraindays
    tmpX = [x_data[i,0-steps:] for i in train_days_list]
    tmpY = [y_data[i,:] for i in train_days_list]
    X = np.array(tmpX).reshape(n_batch,steps,feature_size)
    Y = np.array(tmpY).reshape(n_batch,feature_size)
    return (X,Y)

In [4]:
def test_data_gen(x_data,y_data,steps = 48, n_batch = test_batch_size):
    X = np.zeros((n_batch,steps,feature_size))
    Y = np.zeros((n_batch,feature_size))
    #print x_data[:,0-steps:].shape,y_data.shape
    #print n_batch, steps
    X = x_data[:,0-steps:].reshape(n_batch,steps,feature_size)
    Y = y_data.reshape(n_batch,feature_size)
    return (X,Y)

### Step 5: construct RNN model

In [5]:
# create placeholder for x and y
x = tf.placeholder("float",[None,n_steps,feature_size])
istate = tf.placeholder("float",[None,num_layers*2*n_hidden])
y = tf.placeholder("float",[None,n_output])


# Define weights
weights = {
    'hidden': tf.Variable(tf.random_normal([feature_size, n_hidden])), # Hidden layer weights
    'out': tf.Variable(tf.random_normal([n_hidden, n_output]))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_output]))
}

In [6]:
def RNN(_X, _istate, _weights, _biases):

    # input shape: (batch_size, n_steps, n_input)
    _X = tf.transpose(_X, [1, 0, 2])  # permute n_steps and batch_size
    # Reshape to prepare input to hidden activation
    _X = tf.reshape(_X, [-1, feature_size]) # (n_steps*batch_size, n_input)
    # Linear activation
    _X = tf.matmul(_X, _weights['hidden']) + _biases['hidden']

    # Define a lstm cell with tensorflow
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)
    stacked_lstm_cell = rnn_cell.MultiRNNCell([lstm_cell]*num_layers)
    
    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _X = tf.split(0, n_steps, _X) # n_steps * (batch_size, n_hidden)

    # Get lstm cell output
    outputs, states = rnn.rnn(stacked_lstm_cell, _X, initial_state=_istate)

    # Linear activation
    # Get inner loop last output
    return tf.matmul(outputs[-1], _weights['out']) + _biases['out']

In [ ]:
pred = RNN(x, istate, weights, biases)

#cost function 
cost = tf.reduce_mean(tf.pow(pred-y,2)) # cost function of this batch of data
cost2 = tf.abs(pred-y) # 
#compute parameter updates
#train_op = tf.train.GradientDescentOptimizer(0.008).minimize(cost)
optimizer = tf.train.RMSPropOptimizer(0.005, 0.3).minimize(cost)
optimizer2 = tf.train.RMSPropOptimizer(0.005, 0.3).minimize(cost2)

In [ ]:
## iterating among all customers to find current training customer
result_final = []
for i in range(75,200):
    test_x_name = data_dir + 'test_x_' + str(i) + '.csv'
    test_y_name = data_dir + 'test_y_' + str(i) + '.csv'
    train_x_name = data_dir + 'train_x_' + str(i) + '.csv'
    train_y_name = data_dir + 'train_y_' + str(i) + '.csv'
    tmp_data = np.array(pd.read_csv(test_x_name,header = None))
    test_x_data = tmp_data[:,1:]
    # print test_x_data.dtype  data are stored as float64 double precision format
    tmp_data = np.array(pd.read_csv(test_y_name,header = None))
    test_y_data = tmp_data[:,1:]
    tmp_data = np.array(pd.read_csv(train_x_name,header = None))
    train_x_data = tmp_data[:,1:]
    tmp_data = np.array(pd.read_csv(train_y_name,header = None))
    train_y_data = tmp_data[:,1:]
    traindays = train_y_data.shape[0]
    # generate test data
    test_x,test_y = test_data_gen(test_x_data,test_y_data,n_steps)
    test_x = test_x.reshape(test_batch_size,n_steps,feature_size)
    ### Execute
    # Initializing the variables
    init = tf.initialize_all_variables()
    outp = []
    outlist = np.zeros([Rs,test_batch_size])
    with tf.Session() as sess:
        # Create a summary to monitor cost function
        #tf.scalar_summary("loss", cost)
        #tf.scalar_summary("loss2",cost2)
        # Merge all summaries to a single operator
        #merged_summary_op = tf.merge_all_summaries()

        # tensorboard info.# Set logs writer into folder /tmp/tensorflow_logs
        #summary_writer = tf.train.SummaryWriter('/tmp/tensorflow_logs', graph_def=sess.graph_def)

        #initialize all variables in the model
        sess.run(init)
        for k in range(num_epoches):
            #Generate Data for each epoch
            #What this does is it creates a list of of elements of length seq_len, each of size [batch_size,input_size]
            #this is required to feed data into rnn.rnn
            #print traindays
            X,Y = train_data_gen(traindays,train_x_data,train_y_data,n_steps)
            X = X.reshape(train_batch_size,n_steps,feature_size)


            #Create the dictionary of inputs to feed into sess.run
            if k < 50:
                sess.run(optimizer2,feed_dict={x:X,y:Y,istate:np.zeros((train_batch_size,num_layers*2*n_hidden))})
            else:
                sess.run(optimizer,feed_dict={x:X,y:Y,istate:np.zeros((train_batch_size,num_layers*2*n_hidden))})   
            #perform an update on the parameters

            loss1 = sess.run(cost, feed_dict = {x:X,y:Y,istate:np.zeros((train_batch_size,num_layers*2*n_hidden))} )
            loss2 = sess.run(cost, feed_dict = {x:test_x,y:test_y,istate:np.zeros((test_batch_size,num_layers*2*n_hidden))} )            #compute the cost on the validation set
            output_tmp = sess.run(pred,feed_dict = {x:X,y:Y,istate:np.zeros((train_batch_size,num_layers*2*n_hidden))} )
            outp_train = output_tmp
            output_tmp = sess.run(pred,feed_dict = {x:test_x,y:test_y,istate:np.zeros((test_batch_size,num_layers*2*n_hidden))} )
            outp_test = output_tmp
            if k >= num_epoches-Rs:
                outlist[k-num_epoches+Rs,:] = outp_test.copy().T

            # Write logs at every iteration
            #summary_str = sess.run(merged_summary_op, feed_dict={x:test_x,y:test_y,istate:np.zeros((test_batch_size,num_layers*2*n_hidden))} )
            #summary_writer.add_summary(summary_str, k)
            print "Iter " + str(k) + ", Minibatch Loss ---- Train = " + "{:.6f}".format(loss1) + "; Test = " + "{:.6f}".format(loss2)
        #print "haha{}".format(outp)
    R = []
    RR  = []
    for i in range(Rs):
        out = np.array(outlist[i])
        R.append(np.corrcoef(out.T,test_y.T)[0,1])
        RR.append(np.corrcoef(out.T,test_y.T)[0,1]**2)
    print R
    RRR = np.mean(R)# average Rs R in this time of train
    
    
    # run time
    time2 = time.time()
    print 'total running time cost:{}s'.format(time2-time1)
    
    # append R
    result_final.append(RRR)

Iter 0, Minibatch Loss ---- Train = 0.135950; Test = 0.137269
Iter 1, Minibatch Loss ---- Train = 0.198104; Test = 0.176260
Iter 2, Minibatch Loss ---- Train = 0.127573; Test = 0.128228
Iter 3, Minibatch Loss ---- Train = 0.132068; Test = 0.135174
Iter 4, Minibatch Loss ---- Train = 0.121880; Test = 0.128049
Iter 5, Minibatch Loss ---- Train = 0.109803; Test = 0.113796
Iter 6, Minibatch Loss ---- Train = 0.122853; Test = 0.125285
Iter 7, Minibatch Loss ---- Train = 0.105977; Test = 0.100961
Iter 8, Minibatch Loss ---- Train = 0.119875; Test = 0.120535
Iter 9, Minibatch Loss ---- Train = 0.102730; Test = 0.092600
Iter 10, Minibatch Loss ---- Train = 0.115580; Test = 0.114180
Iter 11, Minibatch Loss ---- Train = 0.090399; Test = 0.088776
Iter 12, Minibatch Loss ---- Train = 0.119237; Test = 0.110392
Iter 13, Minibatch Loss ---- Train = 0.091734; Test = 0.084736
Iter 14, Minibatch Loss ---- Train = 0.106921; Test = 0.102086
Iter 15, Minibatch Loss ---- Train = 0.092942; Test = 0.086448
It

In [ ]:
result_final

In [ ]:
# Draw
xxx = np.arange(0,test_batch_size)
pl.plot(xxx,out,color = "red")
pl.plot(xxx,test_y)
pl.grid()
pl.legend()
pl.show()

In [ ]:
out = np.array(outlist[0])
np.corrcoef(out.T,test_y.T)

In [ ]:
DataFrame(result_final).to_csv('result2.csv')

In [ ]:
train_y_data.shape